# Notebook to explain new functionality for the assembly_fragments function

So I love using the assembly_fragments function and think it is super nice and easy to use. However, I have always thought that the assembly_fragments function could have the functionality of choosing which direction primers are made (see examples below). 


For reference the Klaevins lab made Coral https://github.com/klavinslab/coral?tab=readme-ov-file many years ago and they have this functionality in their gibson module: https://coral.readthedocs.io/en/latest/api.html#module-coral.design 


### From Coral 



In [ ]:

'''
coral.design.gibson(seq_list, circular=True, overlaps='mixed', overlap_tm=65, maxlen=80, terminal_primers=True, primer_kwargs=None)

Design Gibson primers given a set of sequences

Parameters:	
seq_list (list containing coral.DNA) – List of DNA sequences to stitch together
circular (bool) – If true, designs primers for making a circular construct. If false, designs primers for a linear construct.
splits (str or list of str) – Specifies locations of overlap. Must be either a single entry of the same type as the ‘split’ parameter in gibson_primers or a list of those types of the appropriate length (for circular construct, len(seq_list), for linear construct, len(seq_list) - 1)
overlap_tm (float) – Minimum Tm of overlap
maxlen (int) – Maximum length of each primer.
terminal_primers (bool) – If the output is not circular, will design non-Gibson primers for amplifying the first and last fragments sans homology. If False, there will be one less set of primers returned.
primer_kwargs (dict) – keyword arguments to pass to design.primer

Returns:	
Forward and reverse primers for amplifying every fragment.

Return type:	
a list of sequence.Primer tuples

Raises:	
ValueError if split parameter is an invalid string or wrong size.
'''

# From Pydna documentation

Example 1: Linear assembly of PCR products (pydna.amplicon.Amplicon class objects)


        >       <         >       <
        Amplicon1         Amplicon3
                 Amplicon2         Amplicon4
                 >       <         >       <

                             ⇣
                             pydna.design.assembly_fragments
                             ⇣

        >       <-       ->       <-                      pydna.assembly.Assembly
        Amplicon1         Amplicon3
                 Amplicon2         Amplicon4     ➤  Amplicon1Amplicon2Amplicon3Amplicon4
                ->       <-       ->       <


Low level perspective: 

      agcctatcatcttggtctctgca
      |||||||||||||||||||||||
                      gagacgtAAATATA

      |||||||||||||||||||||||
      tcggatagtagaaccagagacgt


                             TTTATATCGCATGACTCTTCTTT
                             |||||||||||||||||||||||

                      ctctgcaTTTATAT
                             |||||||||||||||||||||||
                             AAATATAGCGTACTGAGAAGAAA

PCR products with flanking sequences are formed in the PCR process.



      agcctatcatcttggtctctgcaTTTATAT
      ||||||||||||||||||||||||||||||
      tcggatagtagaaccagagacgtAAATATA
                      \\____________/

                         identical
                         sequences
                       ____________
                      /            \\
                      ctctgcaTTTATATCGCATGACTCTTCTTT
                      ||||||||||||||||||||||||||||||
                      gagacgtAAATATAGCGTACTGAGAAGAAA
        
----------------------------------------------------------------
NEW functionality: Linear assembly of PCR products but choose overhang side (pydna.amplicon.Amplicon class objects) ------
Choose how overlapping is formed.
- Could for example be:
1. Right overlap, 
2. 50/50 overlap (like it is now), or 
3. Left overlap.

Right overlapping:
----------------------------------------------------------------

        >       <         >       <
        Amplicon1         Amplicon3
                 Amplicon2         Amplicon4
                 >       <         >       <

                             ⇣
                pydna.design.assembly_fragments
                             ⇣

        >       <--       >       <--                     pydna.assembly.Assembly
        Amplicon1         Amplicon3
                 Amplicon2         Amplicon4     ➤  Amplicon1Amplicon2Amplicon3Amplicon4
                 >       <--       >       <



Low level perspective: 

      agcctatcatcttggtctctgca
      |||||||||||||||||||||||
                      gagacgtAAATATAGCGTACT

      |||||||||||||||||||||||
      tcggatagtagaaccagagacgt


                             TTTATATCGCATGACTCTTCTTT
                             |||||||||||||||||||||||

                             TTTATAT
                             |||||||||||||||||||||||
                             AAATATAGCGTACTGAGAAGAAA

PCR products with flanking sequences are formed in the PCR process.



      agcctatcatcttggtctctgcaTTTATATCGCATGA
      |||||||||||||||||||||||||||||||||||||
      tcggatagtagaaccagagacgtAAATATAGCGTACT
                            \\____________/

                                identical
                                sequences
                               ____________
                             /            \\
                             TTTATATCGCATGACTCTTCTTT
                             |||||||||||||||||||||||
                             AAATATAGCGTACTGAGAAGAAA
        


Left overlapping:
----------------------------------------------------------------

        >       <         >       <
        Amplicon1         Amplicon3
                 Amplicon2         Amplicon4
                 >       <         >       <

                             ⇣
                             pydna.design.assembly_fragments
                             ⇣

        >       <       -->       <                       pydna.assembly.Assembly
        Amplicon1         Amplicon3
                 Amplicon2         Amplicon4     ➤  Amplicon1Amplicon2Amplicon3Amplicon4
               -->       <       -->       <







# example code: 

In [18]:
from pydna.dseqrecord import Dseqrecord
from pydna.design import primer_design
a=primer_design(Dseqrecord("atgactgctaacccttccttggtgttgaacaagatcgacgacatttcgttcgaaacttacgatg"))
b=primer_design(Dseqrecord("ccaaacccaccaggtaccttatgtaagtacttcaagtcgccagaagacttcttggtcaagttgcc"))
c=primer_design(Dseqrecord("tgtactggtgctgaaccttgtatcaagttgggtgttgacgccattgccccaggtggtcgtttcgtt"))
from pydna.design import assembly_fragments
# We would like a circular recombination, so the first sequence has to be repeated
fa1,fb,fc,fa2 = assembly_fragments([a,b,c,a]) # adding the overlap_side  argument (str) – Specifies location of overlap. ‘left’ puts it on the ‘dna1’ side (i.e. the primer to amplify dna2). ‘right’ puts it on the dna2 side, and ‘mixed’ does a ~50:50 split



In [19]:
# Since all fragments are Amplicons, we need to extract the rp of the 1st and fp of the last fragments.
from pydna.amplify import pcr
fa = pcr(fa2.forward_primer, fa1.reverse_primer, a)
fa.name, fb.name, fc.name = "fa fb fc".split()
from pydna.assembly import Assembly
assemblyobj = Assembly([fa,fb,fc])
assemblyobj

Assembly
fragments..: 100bp 101bp 102bp
limit(bp)..: 25
G.nodes....: 6
algorithm..: common_sub_strings

In [20]:
assemblyobj.assemble_linear()

[Contig(-231), Contig(-166), Contig(-36)]

In [21]:
print(assemblyobj.assemble_circular()[0].figure())

 -|fa|36
|     \/
|     /\
|     36|fb|36
|           \/
|           /\
|           36|fc|36
|                 \/
|                 /\
|                 36-
|                    |
 --------------------
